In [1]:
import pandas as pd
import numpy as np
import pprint
from sklearn.cluster import DBSCAN
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.options.display.max_columns = 500
pd.options.display.max_rows = 200 
pd.options.display.max_colwidth = 100

# Exploration

In [2]:
data = pd.read_csv('../data/processed/data.csv')

In [3]:
def get_context(data, symbol, columns = data.columns.tolist()):
    idx_mask = data.symbol == symbol
    return data.loc[idx_mask, columns]

In [7]:
context=[
    'symbol', 
    'calendarYear', 
    'fillingDate',
    'target', 
    'revenue',
    'totalAssets', 
    'netIncome', 
    'totalEquity',
    'totalLiabilitiesAndTotalEquity',
    'finalLink'
    ]

In [46]:
get_context(data, 'CLX', context)

,symbol,calendarYear,fillingDate,target,revenue,totalAssets,netIncome,totalEquity,totalLiabilitiesAndTotalEquity,finalLink
1809,CLX,1996,1996-09-26,8.04820,3.53433,3.47234,0.35394,1.48653,3.47234,https://www.sec.gov/Archives/edgar/data/21076/0000021076-96-000004.txt
1810,CLX,1997,1997-09-25,12.00949,3.95825,4.34162,0.38978,1.61912,4.34162,https://www.sec.gov/Archives/edgar/data/21076/0000021076-97-000008.txt
1811,CLX,1998,1998-09-28,15.68516,4.24269,4.68950,0.46121,1.67955,4.68950,https://www.sec.gov/Archives/edgar/data/21076/0000021076-98-000006.txt
1812,CLX,1999,1999-09-28,14.91446,6.11003,6.30693,0.37548,2.39639,6.30693,https://www.sec.gov/Archives/edgar/data/21076/0000021076-99-000008.txt
1813,CLX,2000,2000-09-26,14.03428,6.12541,6.53047,0.59109,2.69140,6.53047,https://www.sec.gov/Archives/edgar/data/21076/000002107600000003/0000021076-00-000003-0001.txt
1814,CLX,2001,2001-09-28,12.90082,5.73535,5.87054,0.47464,2.79200,5.87054,https://www.sec.gov/Archives/edgar/data/21076/000095014901501457/f75998e10-k.txt
1815,CLX,2002,2002-09-26,13.48277,5.87393,5.25052,0.46575,1.95846,5.25052,https://www.sec.gov/Archives/edgar/data/21076/000002107602000019/tidiedtext.htm
1816,CLX,2003,2003-09-19,14.01202,5.90380,5.20287,0.70236,1.73096,5.20287,https://www.sec.gov/Archives/edgar/data/21076/000002107603000124/text.htm
1817,CLX,2004,2004-08-27,15.93420,6.03301,5.34934,0.76599,2.14867,5.34934,https://www.sec.gov/Archives/edgar/data/21076/000002107604000063/f10ktext.htm
1818,CLX,2005,2005-08-31,11.96488,5.99720,4.94345,1.49793,-0.75580,4.94345,https://www.sec.gov/Archives/edgar/data/21076/000110465905042311/a05-15203_110k.htm


In [10]:
data[(data.shortTermDebt==0)&(data.longTermDebt == 0)]

,symbol,calendarYear,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,goodwillAndIntangibleAssets,longTermInvestments,taxAssets,otherNonCurrentAssets,totalNonCurrentAssets,otherAssets,totalAssets,accountPayables,shortTermDebt,taxPayables,deferredRevenue,otherCurrentLiabilities,totalCurrentLiabilities,longTermDebt,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,otherNonCurrentLiabilities,totalNonCurrentLiabilities,otherLiabilities,capitalLeaseObligations,totalLiabilities,preferredStock,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalEquity,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,revenue,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestIncome,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,date,reportedCurrency,cik,fillingDate,acceptedDate,period,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink,month,inflationMultiplier,mcapDate,mcapYear,mcapWeek,target,brent,chicagoFedFinancialConditions,consumerSentiment,corePCE,GDP,interest10Y,interest10Y3M,interest10YIInflationAdjusted,interest20Y,interest3M,inventorySalesRatio,leadingIndex,monthlySupplyHouses,moodysAaa20Y,moodysBaa20Y,mortgage30Y,nasdaq,stlouisFredFinancialStress,unemployment,volatility,wilshire5000,wti
5,A,2005,3.02004,0.03392,3.05396,1.02160,0.97955,0.97819,6.03330,1.18441,0.00000,0.49113,0.49113,0.00000,0.00000,1.45033,3.12587,0.00000,9.15917,0.46671,0.00000,0.00000,0.33511,1.82478,2.62660,0.00000,0.00000,0.00000,0.99583,0.99583,0.00000,0.00000,3.62242,0.00000,0.00678,-1.98487,-0.06648,7.58131,5.53674,9.15917,0.00000,5.53674,9.15917,0.00000,0.00000,-3.02004,6.97215,3.55052,3.42163,0.00000,1.00125,2.17481,0.00000,2.17481,0.00000,3.17606,6.72658,0.00000,0.00000,0.25235,0.90628,0.00000,0.24556,0.00000,0.09904,0.34460,0.00000,0.21029,0.44365,0.00000,0.00000,0.00000,0.67022,0.67836,2005-10-31,USD,1090872.00000,2006-01-17,2006-01-17 16:47:54,FY,-0.01628,0.00000,-0.26456,0.00000,0.00000,3.40671,0.80453,1.21968,-0.18858,-0.08683,-0.05156,0.04070,0.01492,-0.27134,0.00000,0.26863,-0.39345,0.00000,-0.93342,-1.05824,-0.01085,-0.12075,3.02004,3.14079,1.21968,-0.18858,1.03110,https://www.sec.gov/Archives/edgar/data/1090872/000104746906000516/a2166540z10-k.htm,https://www.sec.gov/Archives/edgar/data/1090872/000104746906000516/a2166540z10-k.htm,10.00000,1.35671,2006-01-27,2006.00000,4.00000,16.05531,63.22000,-0.61163,88.87742,89.57732,13626.58471,4.34000,-0.04000,1.95000,4.57000,4.28000,1.25516,1.73129,5.71290,5.21000,6.17000,6.11429,2302.69000,-0.90329,4.75161,11.91000,12896.94000,66.36000
44,AAPL,2004,4.12824,3.46917,7.59742,1.45997,0.14044,0.61180,9.80962,0.98305,0.11124,0.02364,0.13487,0.00000,0.11958,0.14600,1.38350,0.00000,11.19312,2.01754,0.00000,0.00000,0.75640,0.95246,3.72640,0.00000,0.00000,0.40879,0.00000,0.40879,

In [6]:
data[data.totalAssets == 0]

,symbol,calendarYear,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,goodwillAndIntangibleAssets,longTermInvestments,taxAssets,otherNonCurrentAssets,totalNonCurrentAssets,otherAssets,totalAssets,accountPayables,shortTermDebt,taxPayables,deferredRevenue,otherCurrentLiabilities,totalCurrentLiabilities,longTermDebt,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,otherNonCurrentLiabilities,totalNonCurrentLiabilities,otherLiabilities,capitalLeaseObligations,totalLiabilities,preferredStock,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalEquity,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,revenue,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestIncome,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,date,reportedCurrency,cik,fillingDate,acceptedDate,period,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink,month,inflationMultiplier,mcapDate,mcapYear,mcapWeek,target


In [7]:
data[data.revenue == 0]

,symbol,calendarYear,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,goodwillAndIntangibleAssets,longTermInvestments,taxAssets,otherNonCurrentAssets,totalNonCurrentAssets,otherAssets,totalAssets,accountPayables,shortTermDebt,taxPayables,deferredRevenue,otherCurrentLiabilities,totalCurrentLiabilities,longTermDebt,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,otherNonCurrentLiabilities,totalNonCurrentLiabilities,otherLiabilities,capitalLeaseObligations,totalLiabilities,preferredStock,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalEquity,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,revenue,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestIncome,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,date,reportedCurrency,cik,fillingDate,acceptedDate,period,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink,month,inflationMultiplier,mcapDate,mcapYear,mcapWeek,target


###  DBSCAN for outliers??

In [ ]:
dbscan = DBSCAN(eps=75, min_samples=5)
dbscan.fit(data)

DBSCAN(eps=75)

# Maybe check increases

In [ ]:
also_compute = [
    'weightedAverageShsOut',
    'weightedAverageShsOutDil',
    'preferredStock',
    'commonStock',
]
for col in currency_features + also_compute:
    yoy_colname = col + '_yoy'
    data[yoy_colname] = data[col].pct_change(1)